In [34]:
from hyperparams import Hyperparams as hp
import en2divtrans
import io
import numpy as np

source_line = en2divtrans.LoadVocab(hp.source_file)
target_line = en2divtrans.LoadVocab(hp.target_file)
max_word_len = hp.max_seq_len

model = en2divtrans.MakeModel(hidden_unit=hp.hidden, max_len=hp.max_seq_len, x_vocab=len(source_line), y_vocab=len(target_line), embedding_dim=hp.char_emb_dim)
model.load_weights(hp.model_name+'.h5')

def Encode(bas):
    x = []
    temp_x = []
    source_str = bas.strip()
    for i in range(len(source_str)):
        temp_x.append(source_line.index(source_str[i]))
    r = max_word_len - len(source_str)
    for i in range(r):
        temp_x.append(0)
    x.append(temp_x)
    return x

def Predict(x):
    x = np.reshape(x, (len(x), max_word_len))
    y = model.predict(x=x, batch_size=hp.batch, verbose=0)
    
    return (y,x)

def Decode(y,x):
    for i in range(len(y)):
        yw = y[i]
        xw = x[i]
        strs_x = ''
        for j in range(len(xw)):
            if source_line[xw[j]] != '<P>':
                strs_x += source_line[xw[j]]
        strs_x = strs_x.strip()
        strs_y = ''
        for j in range(len(yw)):
            c = target_line[np.argmax(yw[j], axis=0)]
            if c != '<P>':
                strs_y +=c
        strs_y = strs_y.strip()
        
    return (strs_y)

def LatinToThaana(string):
    latinbas = Encode(string)
    pred,x = Predict(latinbas)
    thaanabas = Decode(pred,x)
    
    return (thaanabas)
        
if __name__ == '__main__':
    latin = "MDP majlis membarunnaa MDP ah haradhu kuraa meehun MMPRC in kalhu faisaa nagaafa thibumakee MDP JP adhi adhaalthu PPM aai evvaru vun"
    latin = latin.lower()
    basthah = latin.split(" ")
    thaanajumla =""
    for r in range (len(basthah)):
        thaanabas = LatinToThaana(basthah[r])
        thaanajumla = (thaanajumla + " " + thaanabas)

    print (thaanajumla.strip())

    

model summary...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_x_char_idx (InputLayer (None, 35)                0         
_________________________________________________________________
embedding_33 (Embedding)     (None, 35, 64)            3456      
_________________________________________________________________
bidirectional_33 (Bidirectio (None, 35, 128)           197632    
_________________________________________________________________
output_y_softmax (TimeDistri (None, 35, 57)            7353      
Total params: 208,441
Trainable params: 208,441
Non-trainable params: 0
_________________________________________________________________
އެމްޑީޕީ މަޖިލިސް މެމްބަރުންނާ އެމްޑީޕީ އަށް ހަރަދު ކުރާ މީހުން އެމްއެމްޕީއާރްސީ އިން ކަޅު ފައިސާ ނަގާފަ ތިބުމަކީ އެމްޑީޕީ ޖޭޕީ އަދި އަދާލުތު ޕީޕީއެމް އާއި އެއްވަރު ވުންް
